In [ ]:
!pip3 install dill
!pip3 install kaleido
import pandas as pd
import numpy as np
import dill as pickle
import plotly.express as px
import plotly.graph_objects as go
import kaleido

from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
df = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/!main/dataset/final_without_fill_na_salary.csv')
df.head()

## построим график по скиллам

In [ ]:
skill_df = df.copy()
skill_df.Skills = skill_df.Skills.str.split(',')
skill_df = skill_df.explode('Skills')
skill_df.Skills = skill_df.Skills.str.strip()

skills = skill_df.Skills.value_counts().head(20)
skills

Python              812
SQL                 492
Git                 325
PostgreSQL          322
Linux               232
Django Framework    196
Английский язык     130
Docker              130
Анализ данных        89
Flask                79
Django               65
Data Analysis        64
Redis                61
ETL                  57
RabbitMQ             56
С                    56
Pandas               54
JavaScript           53
Machine Learning     53
Big Data             52
Name: Skills, dtype: int64

In [ ]:
skillpiv = skill_df.pivot_table(index = 'Skills', columns='Category', aggfunc='size', fill_value=0).loc[skills.index]
skillpiv

In [ ]:
#если просто сделать из датафрейма to_dict() воспринимается ок

skills_to_dash = {"most_common_skills": skillpiv.to_dict()}
with open('/drive/MyDrive/!SberDataScience/Курсовой проект/!main/models/dash0', 'wb') as file:
    pickle.dump(skills_to_dash, file)
    

In [ ]:
fig = px.bar(skillpiv.to_dict())
fig.update_layout(
    title="Most frequent skills",
    xaxis_title="Skills",
    yaxis_title="Count",
    legend_title="Category",
    
)

#fig.show()
#fig.write_image('skills.png', scale=2) #сохранение в png, для этого обязательно установить kaleido
#fig.write_html('skills.html') #сохранение в html

## сколько в датасете каких позиций

In [ ]:
vacancy = round(df.Category.value_counts(normalize=True)*100, 2)

In [ ]:
vac_to_dash = {"most_common_vac": vacancy.to_dict()}
with open('/drive/MyDrive/!SberDataScience/Курсовой проект/!main/models/dash1', 'wb') as file:
    pickle.dump(vac_to_dash, file)

In [ ]:
#код распаковки словаря для пирога

nvac = pd.DataFrame.from_dict(vacancy.to_dict(), orient='index')
labels = nvac.index
values = nvac.values.flatten()

In [ ]:
fig = px.pie(labels=labels, values=values, names=labels, title="Distribution of vacancy fields")
fig.show()

#fig.write_image('vacancy.png', scale=2) #сохранение в png, для этого обязательно установить kaleido
#fig.write_html('vacancy.html') #сохранение в html

## Средниее зп по категориям

In [ ]:
salary = df[['Category', 'Salary_median']].copy()

In [ ]:
#пикл 
salary_to_dash = {"salary": salary.to_dict()}
with open('/drive/MyDrive/!SberDataScience/Курсовой проект/!main/models/dash2', 'wb') as file:
    pickle.dump(salary_to_dash, file)

In [ ]:
fig = px.box(salary.to_dict(), x="Category", y="Salary_median")
fig.update_layout(
    title="Salary according to vacancy",
    xaxis_title="Vacancy",
    yaxis_title="Salary"
    
)
fig.show()

#fig.write_image('salary.png', scale=2) #сохранение в png, для этого обязательно установить kaleido
#fig.write_html('salary.html') #сохранение в html

## Зависимость зп от опыта работы

In [ ]:
exp = df[['Category', 'Experience', 'Salary_median']].copy()
exp.Experience = pd.Categorical(exp.Experience, categories=['Нет опыта', 'От 1 года до 3 лет', 'От 3 до 6 лет', 'Более 6 лет'], ordered=True)

In [ ]:
exppiv = exp.pivot_table(index='Experience', columns='Category', values='Salary_median', aggfunc='mean')

#пикл 
exp_to_dash = {"experience": exppiv.to_dict()}
with open('/drive/MyDrive/!SberDataScience/Курсовой проект/!main/models/dash3', 'wb') as file:
    pickle.dump(exp_to_dash, file)

In [ ]:
fig = px.line(exppiv.to_dict())
fig.update_layout(
    title="Salary according to experience",
    xaxis_title="Years of experience",
    yaxis_title="Salary",
    legend_title="Vacancy"
    
)
fig.show()

#fig.write_image('experience.png', scale=2) #сохранение в png, для этого обязательно установить kaleido
#fig.write_html('experience.html') #сохранение в html

## Зависимость зп от уровня позиции

In [ ]:
position = df[['Category', 'Position_level', 'Salary_median']].copy()
position.drop(df[df.Position_level=='other'].index, inplace=True)
position.Position_level = pd.Categorical(position.Position_level, categories=['Junior', 'Middle', 'Senior', 'Lead'], ordered=True)

In [ ]:
pospiv= position.pivot_table(index='Position_level', columns ='Category', values='Salary_median', aggfunc='mean')

In [ ]:
#пикл 
level_to_dash = {"level": pospiv.to_dict()}
with open('/drive/MyDrive/!SberDataScience/Курсовой проект/!main/models/dash4', 'wb') as file:
    pickle.dump(level_to_dash, file)

In [ ]:
fig = px.line(pospiv.to_dict())
fig.update_layout(
    title="Salary according to position level",
    xaxis_title="Position level",
    yaxis_title="Salary",
    legend_title="Vacancy"
    
)
fig.show()

#fig.write_image('position.png', scale=2) #сохранение в png, для этого обязательно установить kaleido
#fig.write_html('position.html') #сохранение в html